# Running the Tweeted Configurations

This notebook runs the latest tweeted battery maodel configuration, and can also be modified and played with. The only input required by the user is the varied parameter's value, if any parameter was varied. Everything else requires no user input (unless you want to modify some stuff)

In [1]:
%pip install git+https://github.com/pybamm-team/PyBaMM@develop  # install the develop branch of PyBaMM
import os
import pybamm
import imageio
import logging
import numpy as np
import matplotlib.pyplot as plt


# looging configs
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\users\saransh\saransh_softwares\python_3.9\python.exe -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


We start by opening config.txt, where the most recent tweeted configuration is stored. Next we read that data and convert it into a python dictionary.

In [2]:
# open the file and read the python dictionary for configurations
f = open('config.txt', 'r')
config = eval(f.read())

# if a parameter's value is varied then taking it the from user

# please enter the varied values in this list, if not entered the
# the code will give an undesired output.
# Note: You need to enter them only if they are present in the latest
# tweeted plot 
param_values = []

# extracting the models from "config" dictionary
models = []
if "DFN" in config["model"]:
    models.append(pybamm.lithium_ion.DFN(
        options=config["model options"]
    ))
if "spm.SPM" in config["model"]:
    models.append(pybamm.lithium_ion.SPM(
        options=config["model options"]
    ))
if "spme.SPMe" in config["model"]:
    models.append(pybamm.lithium_ion.SPMe(
        options=config["model options"]
    ))

# closing the file
f.close()
config

{'model': '{0: <pybamm.models.full_battery_models.lithium_ion.spme.SPMe object at 0x00000266D3F4CF40>, 1: <pybamm.models.full_battery_models.lithium_ion.dfn.DFN object at 0x00000266D4583E50>}',
 'model options': None,
 'chemistry': {'chemistry': 'lithium_ion',
  'cell': 'kokam_Marquis2019',
  'negative electrode': 'graphite_mcmb2528_Marquis2019',
  'separator': 'separator_Marquis2019',
  'positive electrode': 'lico2_Marquis2019',
  'electrolyte': 'lipf6_Marquis2019',
  'experiment': '1C_discharge_from_full_Marquis2019',
  'sei': 'example',
  'citation': 'Marquis2019'},
 'is_experiment': False,
 'cycle': None,
 'number': None,
 'is_comparison': True,
 'param_to_vary': None}

The following cell is meant to plot summary variables, if they weren't tweeted in the latest then you can skip this. (Note: running the cell won't make a difference if the last tweet was not about summary variables)

In [3]:
pybamm.set_logging_level("NOTICE")

# if summary variables were tweeted
if config["is_experiment"] and not config["is_comparison"]:

    # don't terminate early if Ai2020 parameter sets were used
    if config["chemistry"] == pybamm.parameter_sets.Ai2020:
        experiment = pybamm.Experiment(
            config["cycle"] * config["number"]
        )
    else:
        experiment = pybamm.Experiment(
            config["cycle"] * config["number"], termination="80% capacity"
        )

    # defining parameter_values
    parameter_values = pybamm.ParameterValues(chemistry=config["chemistry"])

    # simulating
    sim = pybamm.Simulation(
        model=models[0],
        experiment=experiment,
        parameter_values=parameter_values,
        solver=pybamm.CasadiSolver(mode="safe"),
    )

    # solving
    if config["chemistry"] == pybamm.parameter_sets.Ai2020:
        sim.solve(calc_esoh=False)
    else:
        sim.solve()
    solution = sim.solution

    # defining the summary variables
    if config["chemistry"] == pybamm.parameter_sets.Ai2020:
        vars_to_plot = [
            "Measured capacity [A.h]",
            "Loss of lithium inventory [%]",
            "Loss of active material in negative electrode [%]",
            "Loss of active material in positive electrode [%]",
        ]
    else:
        vars_to_plot = [
            "Capacity [A.h]",
            "Loss of lithium inventory [%]",
            "Loss of active material in negative electrode [%]",
            "Loss of active material in positive electrode [%]",
            "x_100",
            "x_0",
            "y_100",
            "y_0",
        ]

    length = len(vars_to_plot)
    n = int(length // np.sqrt(length))
    m = int(np.ceil(length / n))

    # plotting the summary variables
    fig, axes = plt.subplots(n, m, figsize=(15, 8))
    for var, ax in zip(vars_to_plot, axes.flat):
        ax.plot(
            solution.summary_variables["Cycle number"],
            solution.summary_variables[var]
        )
        ax.set_xlabel("Cycle number")
        ax.set_ylabel(var)
        ax.set_xlim([1, solution.summary_variables["Cycle number"][-1]])
    fig.tight_layout()
    plt.show()


For the GIFs, we start by defining a function to create all the PNGs and the final GIF.

In [4]:
def plot_graph(solution=None, sim=None, labels=None):
    """
    This function generates 80 plots over a time
    span of t_eval seconds and then compiles them to
    create a GIF.
    Parameters:
        solution: pybamm.Simulation.solution or list
            default: None
        sim: pybamm.Simulation or list
            default: None
        labels: list
            default: None
            A list of labels for the GIF.
    """

    # generating time to plot the simulation
    if solution is not None:
        t = solution["Time [s]"]
        final_time = int(t.entries[len(t.entries) - 1])
        time_array = np.linspace(int(t.entries[0]), final_time, num=80)
    else:
        time_array = np.linspace(0, 3700, num=80)

    images = []
    image_files = []

    output_variables = [
        "Negative particle surface concentration [mol.m-3]",
        "Electrolyte concentration [mol.m-3]",
        "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
        "Negative electrode potential [V]",
        "Electrolyte potential [V]",
        "Positive electrode potential [V]",
        "Terminal voltage [V]",
    ]

    # plotting 80 plots
    for val in time_array:
        plot = pybamm.QuickPlot(
            sim,
            time_unit="seconds",
            labels=labels,
            output_variables=output_variables,
        )
        plot.plot(val)
        images.append("plot" + str(val) + ".png")
        plot.fig.savefig("plot" + str(val) + ".png", dpi=300)
        plt.close()

    # compiling all the plots to create a GIF
    for image in images:
        image_files.append(imageio.imread(image))
    imageio.mimsave('plot.gif', image_files, duration=0.1)

    for image in images:
        os.remove(image)


The following cell is meant to create a GIF for the configurations with no experiment, you can skip this if the latest tweeted GIF had no experiment. (Note: running the cell won't make a difference if the last tweet was not about comparing models with no experiment and with different configurations)

In [5]:
# if it is a comparison plot but has no experiment
if config["is_comparison"] and not config["is_experiment"]:

    # declaring the parameter values
    parameter_values = pybamm.ParameterValues(chemistry=config["chemistry"])
    param_list = []
    labels = []

    # if the param_value list is populated then vary the parameter
    for i in range(0, len(param_values)):
        # copy the original values and append them in the list
        param_list.append(parameter_values.copy())

        # change a parameter value
        param_list[i][
            config["param_to_vary"]
        ] = param_values[i]

        logger.info(
            config["param_to_vary"] + ": " + str(param_values[i])
        )

        labels.append(config["param_to_vary"] + ": " + str(param_values[i]))

        # find the minimum value if "Current function [A]" is varied
        if config["param_to_vary"] == "Current function [A]":
            if config["param_to_vary"] < min_param_value:
                min_param_value = config["param_to_vary"]

    # convert everything to a dict for BatchStudy
    parameter_values_for_comp = dict(
        list(enumerate(param_list))
    )
    models_for_comp = dict(list(enumerate(models)))
    parameter_values_for_comp = dict(list(enumerate([parameter_values])))

    # using BatchStudy for the comparison plots
    s = pybamm.BatchStudy(
            models=models_for_comp,
            parameter_values=parameter_values_for_comp,
            permutations=True,
    )

    # changing the t_eval if "Current function [A]" is being varied
    if config["param_to_vary"] == "Current function [A]":
            factor = min_param_value / paramameter_values[config["param_to_vary"]]
            t_end = (1 / factor * 1.1) * 3600
    else:
        # default t_end
        t_end = 3700

    # solving
    s.solve([0, t_end])

    # find the max "Time [s]" from all the solutions for the GIF
    max_time = 0
    solution = s.sims[0].solution
    for sim in s.sims:
        if sim.solution["Time [s]"].entries[-1] > max_time:
            max_time = sim.solution["Time [s]"].entries[-1]
            solution = sim.solution

    # using "plot_graph" for plotting
    if len(labels) == 0:
        plot_graph(
            solution=solution, sim=s.sims
        )
    else:
        plot_graph(
            solution=solution, sim=s.sims, labels=labels
        )


The following cell is meant to create a GIF for the configurations with experiment, you can skip this if the latest tweeted GIF had no experiment. (Note: running the cell won't make a difference if the last tweet was not about comparing a single experiment with different configurations)

In [6]:
# if it is a comparison plot and has an experiment
if config["is_comparison"] and config["is_experiment"]:

    # declaring the parameter values
    parameter_values = pybamm.ParameterValues(chemistry=config["chemistry"])
    param_list = []
    labels = []

    # if the param_value list is populated then vary the parameter
    for i in range(0, len(param_values)):
        # copy the original values and append them in the list
        param_list.append(parameter_values.copy())

        # change a parameter value
        param_list[i][
            config["param_to_vary"]
        ] = param_values[i]

        logger.info(
            config["param_to_vary"] + ": " + str(param_values[i])
        )

        labels.append(config["param_to_vary"] + ": " + str(param_values[i]))

    # convert everything to a dict for BatchStudy
    parameter_values_for_comp = dict(
        list(enumerate(param_list))
    )
    models_for_comp = dict(list(enumerate(models)))
    experiment = dict(
        list(
            enumerate(
                [
                    pybamm.Experiment(
                        config["cycle"] * config["number"]
                    )
                ]
            )
        )
    )
    
    # using BatchStudy for the comparison plots
    parameter_values_for_comp = dict(list(enumerate([parameter_values])))
    s = pybamm.BatchStudy(
            models=models_for_comp,
            parameter_values=parameter_values_for_comp,
            experiments=experiment,
            permutations=True,
    )

    s.solve()

    # find the max "Time [s]" from all the solutions for the GIF
    max_time = 0
    solution = s.sims[0].solution
    for sim in s.sims:
        if sim.solution["Time [s]"].entries[-1] > max_time:
            max_time = sim.solution["Time [s]"].entries[-1]
            solution = sim.solution

    # using "plot_graph" for plotting
    if len(labels) == 0:
        plot_graph(
            solution=solution, sim=s.sims
        )
    else:
        plot_graph(
            solution=solution, sim=s.sims, labels=labels
        )

There should be a GIF ready in your current working directory once everything executes.

In [7]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/